In [1]:
from typing import Callable

from encoder.base import SpikeEncoder
import optuna

In [2]:
from utils.tuning import (
    create_bsa_encoder,
    create_burst_encoder,
    create_dummy_encoder,
    create_expand_burst_encoder,
    create_expand_phase_encoder,
    create_expand_poisson_encoder,
    create_phase_encoder,
    create_poisson_encoder,
    create_sf_encoder,
    create_tbr_encoder,
)

experiment_list: list[tuple[str, Callable[[optuna.Trial], SpikeEncoder]]] = [
    ("Classifier_Base_Tuning", create_dummy_encoder),
    ("Classifier_Rate_Tuning", create_poisson_encoder),
    ("Classifier_SF_Tuning", create_sf_encoder),
    ("Classifier_TBR_Tuning", create_tbr_encoder),
    ("Classifier_BSA_Tuning", create_bsa_encoder),
    ("Classifier_PE_Tuning", create_phase_encoder),
    ("Classifier_BE_Tuning", create_burst_encoder),
    ("Classifier_Rate_Expand_Tuning", create_expand_poisson_encoder),
    ("Classifier_PE_Expand_Tuning", create_expand_phase_encoder),
    ("Classifier_BE_Expand_Tuning", create_expand_burst_encoder),
]

In [3]:
from utils.tuning import create_objective

from config import DB_CONN_STRING


def create_and_run_study(
    study_name: str,
    create_encoder: Callable[[optuna.Trial], SpikeEncoder],
) -> None:
    """
    Create and run an Optuna study.

    Args:
        study_name (str): The name of the study.
        create_encoder (Callable[[optuna.Trial], SpikeEncoder]): A function to create the encoder.
    """
    objective = create_objective(create_encoder)
    n_trials = 50

    study = optuna.create_study(
        direction="minimize",
        study_name=study_name,
        storage=DB_CONN_STRING,
        load_if_exists=True,
        sampler=optuna.samplers.CmaEsSampler(),
        pruner=optuna.pruners.HyperbandPruner(),
    )
    study.optimize(objective, n_trials=n_trials)

In [5]:
for name, create_encoder in experiment_list:
    create_and_run_study(name, create_encoder)

OperationalError: (psycopg2.OperationalError) connection to server at "10.35.27.75", port 5432 failed: FATAL:  no pg_hba.conf entry for host "10.35.28.5", user "optuna_user", database "optuna_db", SSL encryption
connection to server at "10.35.27.75", port 5432 failed: FATAL:  no pg_hba.conf entry for host "10.35.28.5", user "optuna_user", database "optuna_db", no encryption

(Background on this error at: https://sqlalche.me/e/20/e3q8)